In [15]:
import pandas as pd
from sqlalchemy import create_engine, text

# Remplacez ces valeurs par les informations de votre base de données
db_host = "localhost"
db_name = "olist"
db_user = "postgres"
db_password = "admin"

# Créer une connexion avec la base de données en utilisant sqlalchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")


In [16]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_customers_dataset.csv", encoding="utf-8")
data.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [17]:
zip_codes = data[['customer_zip_code_prefix']].drop_duplicates()
zip_codes.columns = ['zip_code_prefix']

states = data[['customer_state']].drop_duplicates()
states.columns = ['state_code']

cities = data[['customer_city', 'customer_state']].drop_duplicates()
cities.columns = ['city_name', 'state_code']

city_zip_codes = data[['customer_city', 'customer_state', 'customer_zip_code_prefix']].drop_duplicates()
city_zip_codes.columns = ['city_name', 'state_code', 'zip_code_prefix']

customers = data[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix']].drop_duplicates()
customers.columns = ['customer_id', 'customer_unique_id', 'zip_code_prefix']


In [18]:
zip_codes.to_sql('olist_zipcode', engine, if_exists='append', index=False)
states.to_sql('olist_state', engine, if_exists='append', index=False)
cities.to_sql('olist_city', engine, if_exists='append', index=False)
customers.to_sql('olist_customer', engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « olist_zipcode_pkey »
DETAIL:  La clé « (zip_code_prefix)=(14409) » existe déjà.

[SQL: INSERT INTO olist_zipcode (zip_code_prefix) VALUES (%(zip_code_prefix__0)s), (%(zip_code_prefix__1)s), (%(zip_code_prefix__2)s), (%(zip_code_prefix__3)s), (%(zip_code_prefix__4)s), (%(zip_code_prefix__5)s), (%(zip_code_prefix__6)s), (%(zip_code_prefi ... 27589 characters truncated ... e_prefix__996)s), (%(zip_code_prefix__997)s), (%(zip_code_prefix__998)s), (%(zip_code_prefix__999)s)]
[parameters: {'zip_code_prefix__0': 14409, 'zip_code_prefix__1': 9790, 'zip_code_prefix__2': 1151, 'zip_code_prefix__3': 8775, 'zip_code_prefix__4': 13056, 'zip_code_prefix__5': 89254, 'zip_code_prefix__6': 4534, 'zip_code_prefix__7': 35182, 'zip_code_prefix__8': 81560, 'zip_code_prefix__9': 30575, 'zip_code_prefix__10': 39400, 'zip_code_prefix__11': 20231, 'zip_code_prefix__12': 18682, 'zip_code_prefix_

In [ ]:
city_zip_codes

,city_name,state_code,zip_code_prefix
0,franca,SP,14409
1,sao bernardo do campo,SP,9790
2,sao paulo,SP,1151
3,mogi das cruzes,SP,8775
4,campinas,SP,13056
...,...,...,...
99376,paicandu,PR,87145
99379,eugenio de castro,RS,98860
99382,sao paulo,SP,5538
99406,aparecida de goiania,GO,74980


In [ ]:
# Récupérer les city_id à partir de la base de données
cities_db = pd.read_sql("SELECT city_id, city_name, state_code FROM olist_city", engine)

# Joindre les DataFrames
city_zip_codes_joined = city_zip_codes.merge(cities_db, on=['city_name', 'state_code'])

# Insérer les données dans la table city_zip_code
city_zip_codes_joined[['city_id', 'zip_code_prefix']].to_sql('olist_cityzipcode', engine, if_exists='append', index=False)


34